In [1]:
import os
import math
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils_feature_engineering import *

Load data

In [2]:
path = r'D:\NEU\Năm 3\DATA PREP\PROJECT_DATAPREP\GITHUB REPO\dataset\dseb63_final_project_DP_dataset\\'
# train
application_train = pd.read_csv(path + 'dseb63_' + 'application_train.csv')
application_train.drop(columns='Unnamed: 0', inplace=True)
# test
application_test = pd.read_csv(path + 'dseb63_' + 'application_test.csv')
application_test.drop(columns='Unnamed: 0', inplace=True)

In [3]:
# Chỉ giữ lại cột SK_ID_CURR và biến target
application_train_filtered = application_train[['SK_ID_CURR', 'TARGET']]
application_train_filtered

,SK_ID_CURR,TARGET
0,278621,0
1,139008,0
2,138348,0
3,64140,0
4,219374,0
...,...,...
246004,90253,0
246005,9027,0
246006,163598,0
246007,300375,1


In [4]:
# credit_card_balance 
credit_card_balance = pd.read_csv(path + 'dseb63_' + 'credit_card_balance.csv')

Preprocessing data

In [5]:
# Handling SK_DPD & SK_DPD_DEF colums
# Tạo cột "is_DPD" dựa trên điều kiện "SK_DPD > 0"
credit_card_balance["is_DPD"] = (credit_card_balance["SK_DPD"] > 0).astype('int64')

# Tạo cột "is_DPD_DEF" dựa trên điều kiện "SK_DPD_DEF > 0"
credit_card_balance["is_DPD_DEF"] = (credit_card_balance["SK_DPD_DEF"] > 0).astype('int64')

# Loại bỏ cột "SK_DPD" và "SK_DPD_DEF" từ DataFrame
credit_card_balance.drop(columns=["SK_DPD", "SK_DPD_DEF"], inplace=True)

Categorical features

In [6]:
# Get the list of categorical columns
series_type = credit_card_balance.dtypes
list_categorical = series_type[series_type == "object"].index.tolist()
list_categorical


['NAME_CONTRACT_STATUS']

In [7]:
# xây dựng bản đồ phân loại
dict_onehot = {}
for cate in list_categorical:
    list_val = credit_card_balance[cate].value_counts().index.tolist()
    dict_onehot[cate] = list_val

In [8]:
dict_onehot

{'NAME_CONTRACT_STATUS': ['Active',
  'Completed',
  'Signed',
  'Demand',
  'Sent proposal',
  'Refused',
  'Approved']}

In [9]:
# Tạo đặc trưng one-hot encoding từ DataFrame credit_card_balance
df_onehot = gen_one_hot_feat(credit_card_balance, dict_onehot, main_key="SK_ID_CURR")
print(df_onehot.shape)

(3227965, 8)


In [10]:
# Tổng hợp dữ liệu
df_agg01 = agg_common_data(df_onehot, ["max", "sum", "mean"], main_key="SK_ID_CURR")

# Đánh giá đặc trưng của dữ liệu tổng hợp và lưu kết quả vào biến eval_agg01
eval_agg01 = feature_evaluate(application_train_filtered, df_agg01)

{'NAME_CONTRACT_STATUS_Active': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Completed': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Signed': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Demand': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Sent_proposal': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Refused': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Approved': ['max', 'sum', 'mean']}

After agg: (86905, 21)


In [11]:
eval_agg01

,name,auc,corr,coverage
1,NAME_CONTRACT_STATUS_Active_sum,0.553278,-0.057364,1.0
2,NAME_CONTRACT_STATUS_Active_mean,0.517165,0.020913,1.0
5,NAME_CONTRACT_STATUS_Completed_mean,0.513497,-0.021976,1.0
4,NAME_CONTRACT_STATUS_Completed_sum,0.513471,-0.018750,1.0
3,NAME_CONTRACT_STATUS_Completed_max,0.513301,-0.022828,1.0
7,NAME_CONTRACT_STATUS_Signed_sum,0.503422,-0.004908,1.0
6,NAME_CONTRACT_STATUS_Signed_max,0.503385,-0.008785,1.0
8,NAME_CONTRACT_STATUS_Signed_mean,0.503290,-0.001382,1.0
14,NAME_CONTRACT_STATUS_Sent_proposal_mean,0.501399,-0.011211,1.0
12,NAME_CONTRACT_STATUS_Sent_proposal_max,0.501398,-0.010975,1.0


In [12]:
# Lọc các đặc trưng có AUC (Area Under the ROC Curve) nhỏ hơn hoặc bằng 0.501 và đo kích thước kết quả
eval_agg01.query("auc <= 0.501").shape

(10, 4)

In [13]:
# Lựa chọn các đặc trưng với AUC (Area Under the ROC Curve) lớn hơn 0.501 từ kết quả đánh giá
selected_features = eval_agg01.query("auc > 0.501")["name"].tolist()

# Lọc DataFrame df_agg01 để giữ lại chỉ các đặc trưng được lựa chọn
df_agg01 = df_agg01[selected_features]
print(df_agg01.shape)

(86905, 11)


Numerical features

In [14]:
# get list numerical attributes
series_type = credit_card_balance.dtypes
ls_num = series_type[series_type == "int64"].index.tolist()
ls_num = [col for col in ls_num if col not in ["SK_ID_PREV", "SK_ID_CURR"]]
ls_num

['MONTHS_BALANCE',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'CNT_DRAWINGS_CURRENT',
 'is_DPD',
 'is_DPD_DEF']

In [15]:
# Tạo một DataFrame mới chỉ chứa một số cột cụ thể từ DataFrame gốc df_data
df_num = credit_card_balance[["SK_ID_PREV", "SK_ID_CURR"] + ls_num].copy()

df_num.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,CNT_DRAWINGS_CURRENT,is_DPD,is_DPD_DEF
0,2582071,87788,-1,45000,1,0,0
1,2582071,87788,-82,67500,0,0,0
2,2582071,87788,-84,67500,0,0,0
3,2582071,87788,-7,45000,0,0,0
4,2582071,87788,-59,67500,4,0,0


In [16]:
# Đảo ngược dấu của cột "MONTHS_BALANCE" trong DataFrame df_num
df_num["MONTHS_BALANCE"] = df_num["MONTHS_BALANCE"] * -1

In [17]:
# Tổng hợp dữ liệu 
df_agg02 = agg_common_data(df_num[["SK_ID_CURR"] + ls_num], ["max", "min", "sum", "mean", "std"], main_key="SK_ID_CURR")

# Đánh giá đặc trưng của dữ liệu tổng hợp và lưu kết quả vào biến eval_agg02
eval_agg02 = feature_evaluate(application_train_filtered, df_agg02)

{'MONTHS_BALANCE': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_CREDIT_LIMIT_ACTUAL': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_DRAWINGS_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'is_DPD': ['max', 'min', 'sum', 'mean', 'std'],
 'is_DPD_DEF': ['max', 'min', 'sum', 'mean', 'std']}

After agg: (86905, 25)


In [18]:
eval_agg02

,name,auc,corr,coverage
14,CNT_DRAWINGS_CURRENT_std,0.624756,0.108322,0.992637
13,CNT_DRAWINGS_CURRENT_mean,0.624038,0.084266,1.000000
10,CNT_DRAWINGS_CURRENT_max,0.613745,0.101358,1.000000
12,CNT_DRAWINGS_CURRENT_sum,0.596027,0.053405,1.000000
3,MONTHS_BALANCE_mean,0.560254,-0.059817,1.000000
0,MONTHS_BALANCE_max,0.558268,-0.059077,1.000000
2,MONTHS_BALANCE_sum,0.557519,-0.057034,1.000000
4,MONTHS_BALANCE_std,0.556072,-0.058305,0.992637
7,AMT_CREDIT_LIMIT_ACTUAL_sum,0.548749,-0.045471,1.000000
1,MONTHS_BALANCE_min,0.521566,-0.025715,1.000000


Continuous features

In [19]:
# get list continuous attributes
series_type = credit_card_balance.dtypes
ls_con = series_type[series_type == "float64"].index.tolist()
ls_con

['AMT_BALANCE',
 'AMT_DRAWINGS_ATM_CURRENT',
 'AMT_DRAWINGS_CURRENT',
 'AMT_DRAWINGS_OTHER_CURRENT',
 'AMT_DRAWINGS_POS_CURRENT',
 'AMT_INST_MIN_REGULARITY',
 'AMT_PAYMENT_CURRENT',
 'AMT_PAYMENT_TOTAL_CURRENT',
 'AMT_RECEIVABLE_PRINCIPAL',
 'AMT_RECIVABLE',
 'AMT_TOTAL_RECEIVABLE',
 'CNT_DRAWINGS_ATM_CURRENT',
 'CNT_DRAWINGS_OTHER_CURRENT',
 'CNT_DRAWINGS_POS_CURRENT',
 'CNT_INSTALMENT_MATURE_CUM']

In [20]:
# Tạo DataFrame mới chỉ chứa một số cột cụ thể từ DataFrame gốc df_data
df_con = credit_card_balance[["SK_ID_PREV", "SK_ID_CURR"] + ls_con].copy()

print(df_con.shape)

(3227965, 17)


In [21]:
# Tổng hợp dữ liệu 
df_agg03 = agg_common_data(df_con[["SK_ID_CURR"] + ls_con], ["max", "min", "sum", "mean", "std"], main_key="SK_ID_CURR")

# Đánh giá đặc trưng của dữ liệu tổng hợp và lưu kết quả vào biến eval_agg03
eval_agg03 = feature_evaluate(application_train_filtered, df_agg03)


{'AMT_BALANCE': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_DRAWINGS_ATM_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_DRAWINGS_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_DRAWINGS_OTHER_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_DRAWINGS_POS_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_INST_MIN_REGULARITY': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_PAYMENT_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_RECEIVABLE_PRINCIPAL': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_RECIVABLE': ['max', 'min', 'sum', 'mean', 'std'],
 'AMT_TOTAL_RECEIVABLE': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_DRAWINGS_ATM_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_DRAWINGS_OTHER_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_DRAWINGS_POS_CURRENT': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_INSTALMENT_MATURE_CUM': ['max', 'min', 'sum', 'mean', 'std']}

After agg: (86905, 75)


In [22]:
eval_agg03

,name,auc,corr,coverage
13,AMT_DRAWINGS_CURRENT_mean,0.607704,0.058209,1.000000
3,AMT_BALANCE_mean,0.606730,0.088466,1.000000
58,CNT_DRAWINGS_ATM_CURRENT_mean,0.606698,0.105368,0.702327
53,AMT_TOTAL_RECEIVABLE_mean,0.605846,0.087785,1.000000
48,AMT_RECIVABLE_mean,0.605834,0.087769,1.000000
...,...,...,...,...
64,CNT_DRAWINGS_OTHER_CURRENT_std,0.502051,0.012934,0.697351
18,AMT_DRAWINGS_OTHER_CURRENT_mean,0.502022,0.009728,0.702327
19,AMT_DRAWINGS_OTHER_CURRENT_std,0.502016,0.008768,0.697351
16,AMT_DRAWINGS_OTHER_CURRENT_min,0.500023,-0.001665,0.702327


In [23]:
# Lọc các đặc trưng có AUC (Area Under the ROC Curve) nhỏ hơn hoặc bằng 0.501 và đo kích thước kết quả
eval_agg03.query("auc <= 0.501").shape


(2, 4)

Save features

In [24]:
# Kết hợp các DataFrame df_agg01, df_agg02, và df_agg03 thành một DataFrame mới df_feat
df_feat = df_agg01.join(df_agg02).join(df_agg03)

print(df_feat.shape)

(86905, 111)


In [25]:
# Kiểm tra và tạo thư mục nếu chưa tồn tại
features_folder = "features"
if not os.path.exists(features_folder):
    os.makedirs(features_folder)

# Xây dựng đường dẫn cho tệp features của DataFrame df_feat
fname = "credit_card_balance"
fname = os.path.join(features_folder, "{}.pkl.bz2".format(fname))

# Lưu DataFrame df_feat vào tệp được xây dựng
df_feat.to_pickle(fname, compression="bz2")

# In thông báo hoàn thành
print("Lưu trữ các đặc trưng đã hoàn thành!")


Lưu trữ các đặc trưng đã hoàn thành!
